In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import json
import time
import pandas as pd

In [2]:
def get_datasets_from_page(soup):
    # print(soup.prettify())
    datasets = soup.find_all('div', class_='sc-bYpRZF ymler km-listitem--large')
    #print(len(datasets)) 

    dataset_urls = []
    dataset_names = []
    for dataset in datasets:     
        link = dataset.find('a', href=True)['href']         
        full_link = "https://www.kaggle.com" + link
        name = dataset.find('div', class_='sc-blmEgr sc-fGrmBj iVVdBa doCOYX').text.strip()

        dataset_urls.append(full_link)
        dataset_names.append(name)
    
    return dataset_urls, dataset_names

In [3]:

def extract_dataset_description(soup):
    description_div = soup.find('div', class_='sc-fbbrMC eWoIO sc-hTKOPA emvYRc')
    if description_div:
        description = description_div.get_text().strip()
        return description
    else:
        return "Description not found"

In [4]:
# Click the "All Datasets" button to show all datasets
def click_all_datasets_button(driver):
    try:
        all_datasets_button = driver.find_element(By.XPATH, "//button[@aria-label='All datasets']")
        all_datasets_button.click()
        # wait for the data to load
        time.sleep(5)
    except Exception as e:
        print("Could not find or click the All Datasets button.", e)

In [5]:
# start the web driver
driver_path = r"C:\Program Files\Google\Chrome\Application\chromedriver.exe"  # change this to the path to your chromedriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

base_url = "https://www.kaggle.com/datasets"

df = pd.DataFrame({
    'name': [],
    'description': [],
    'url': []
    })

page_num = 1
while True:
    if page_num == 1:
        # for the first page, just use the base URL
        driver.get(base_url)
        click_all_datasets_button(driver)
    else:
        # for subsequent pages, add the page number to the URL
        page_url = f"{base_url}?page={page_num}"
        driver.get(page_url)
    time.sleep(5) 

    # parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # check if there are no more datasets
    no_datasets_element = soup.find("h2", class_="sc-fHejqy sc-bCvmQg ivfcpM OquTh")
    if no_datasets_element:
        print("No more pages available.")
        break

    dataset_urls, dataset_names = get_datasets_from_page(soup)

    # add the datasets to the dataframe
    for i in range(len(dataset_urls)):
        driver.get(dataset_urls[i])
        time.sleep(5)
        dataset_soup = BeautifulSoup(driver.page_source, 'html.parser')
        description = extract_dataset_description(dataset_soup)
        df.loc[len(df)] = [dataset_names[i], description, dataset_urls[i]]

    page_num += 1
    print(f"Moving to page {page_num}")

# close the web driver
driver.quit()

Moving to page 2
Moving to page 3
Moving to page 4
Moving to page 5
Moving to page 6
Moving to page 7
Moving to page 8
Moving to page 9
Moving to page 10
Moving to page 11
Moving to page 12
Moving to page 13
Moving to page 14
Moving to page 15
Moving to page 16
Moving to page 17
Moving to page 18
Moving to page 19
Moving to page 20
Moving to page 21
Moving to page 22
Moving to page 23
Moving to page 24
Moving to page 25
Moving to page 26
Moving to page 27
Moving to page 28
Moving to page 29
Moving to page 30
Moving to page 31
Moving to page 32
Moving to page 33
Moving to page 34
Moving to page 35
Moving to page 36
Moving to page 37
Moving to page 38
Moving to page 39
Moving to page 40
Moving to page 41
Moving to page 42
Moving to page 43
Moving to page 44
Moving to page 45
Moving to page 46
Moving to page 47
Moving to page 48
Moving to page 49
Moving to page 50
Moving to page 51
Moving to page 52
Moving to page 53
Moving to page 54
Moving to page 55
Moving to page 56
Moving to page 57


WebDriverException: Message: unknown error: net::ERR_CONNECTION_ABORTED
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF7F675EEA2+31554]
	(No symbol) [0x00007FF7F66D7ED9]
	(No symbol) [0x00007FF7F659872A]
	(No symbol) [0x00007FF7F6590782]
	(No symbol) [0x00007FF7F6581294]
	(No symbol) [0x00007FF7F6582B02]
	(No symbol) [0x00007FF7F65815C1]
	(No symbol) [0x00007FF7F6580DA1]
	(No symbol) [0x00007FF7F6580CE0]
	(No symbol) [0x00007FF7F657EC6D]
	(No symbol) [0x00007FF7F657F2AC]
	(No symbol) [0x00007FF7F659B501]
	(No symbol) [0x00007FF7F662D33E]
	(No symbol) [0x00007FF7F660D02A]
	(No symbol) [0x00007FF7F662C977]
	(No symbol) [0x00007FF7F660CDD3]
	(No symbol) [0x00007FF7F65DA33B]
	(No symbol) [0x00007FF7F65DAED1]
	GetHandleVerifier [0x00007FF7F6A68B1D+3217341]
	GetHandleVerifier [0x00007FF7F6AB5AE3+3532675]
	GetHandleVerifier [0x00007FF7F6AAB0E0+3489152]
	GetHandleVerifier [0x00007FF7F680E776+750614]
	(No symbol) [0x00007FF7F66E375F]
	(No symbol) [0x00007FF7F66

In [8]:
df.to_csv('kaggle.csv', index=0, encoding='utf-8')